In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import jarque_bera, norm, t, chisquare
from statsmodels.stats.power import tt_ind_solve_power

%matplotlib inline


In [2]:
start_array, stop_array, step_array = 10, 100, 10
start_prob, stop_prob,  step_prob = 0.5, 1.0, 0.01
fisrt_norm, second_norm, student = 0, 3, 3
vyborka = []
params = []
count_size = 1
array = np.arange(start_array, stop_array+step_array, step_array)
prob = np.arange(start_prob, stop_prob + step_prob, step_prob)

file = open("distribution.txt", "w")
distribution_line = []



In [3]:
# generating distribution for every pair of params
for i in (array):
    for j in (prob):
        #TODO: count_size should be changed later - now it's too long to genereta so much samples
        for count in range(count_size):
            rand = np.random.uniform(start_prob, stop_prob)
            if rand > j:
                distr = j*norm(fisrt_norm, second_norm).rvs(i)
            else:
                distr = (1 - j)*t.rvs(student, i)
            vyborka.append(distr)
            params.append(i)
            # distribution_line.append("%s"%(distr))

In [4]:
# file.writelines(distribution_line)

In [7]:
power_jb, p_jb = [], []
power_chi, p_ci = [], []
for i in range(len(vyborka)):
    array = vyborka[i]
    param = params[i]
    jb = jarque_bera(array)
    jb_p = jb[1]
    effective_size = np.mean(array) / np.std(array)
    jb_power = tt_ind_solve_power(effect_size=effective_size, nobs1=param, alpha=jb_p)
    power_jb.append(jb_power)
    p_jb.append(jb_p)
    chi = chisquare(array) #here's a mistake
    chi_p = chi[1]
    chi_power = tt_ind_solve_power(effect_size=effective_size, nobs1=param, alpha=chi_p)
    power_chi.append(chi_power)
    p_ci.append(chi_p)
print("power_jb: ", power_jb)
print("p_jb: ", p_jb)
print("power_chi: ", power_chi)
print("chi_p: ", chi_p)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  


IndexError: tuple index out of range